# 基于本地知识的 ChatGLM 应用实现  
## 相关链接  
原项目Github：https://github.com/imClumsyPanda/langchain-ChatGLM  

ChatGLM-6B 在 ModelWhale 平台的部署与微调教程：https://www.heywhale.com/mw/project/6436d82948f7da1fee2be59e  

## ModelWhale 运行配置  
计算资源：V100 Tensor Core GPU  
镜像：Python3.9 Cuda11.6 Torch1.12.1 官方镜像  

## 介绍  

🤖️ 一种利用 [ChatGLM-6B](https://github.com/THUDM/ChatGLM-6B) + [langchain](https://github.com/hwchase17/langchain) 实现的基于本地知识的 ChatGLM 应用。  

💡 受 [GanymedeNil](https://github.com/GanymedeNil) 的项目 [document.ai](https://github.com/GanymedeNil/document.ai) 和 [AlexZhangji](https://github.com/AlexZhangji) 创建的 [ChatGLM-6B Pull Request](https://github.com/THUDM/ChatGLM-6B/pull/216) 启发，建立了全部基于开源模型实现的本地知识问答应用。  

✅ 本项目中 Embedding 选用的是 [GanymedeNil/text2vec-large-chinese](https://huggingface.co/GanymedeNil/text2vec-large-chinese/tree/main)，LLM 选用的是 [ChatGLM-6B](https://github.com/THUDM/ChatGLM-6B)。依托上述模型，本项目可实现全部使用**开源**模型**离线私有部署**。  

⛓️ 本项目实现原理如下图所示，过程包括加载文件 -> 读取文本 -> 文本分割 -> 文本向量化 -> 问句向量化 -> 在文本向量中匹配出与问句向量最相似的`top k`个 -> 匹配出的文本作为上下文和问题一起添加到`prompt`中 -> 提交给`LLM`生成回答。  

![实现原理图](https://raw.githubusercontent.com/imClumsyPanda/langchain-ChatGLM/master/img/langchain%2Bchatglm.png)  

🚩 本项目未涉及微调、训练过程，但可利用微调或训练对本项目效果进行优化。  

## 本项目使用方式  
核心部分代码为  
```python  
# 执行初始化  
init_cfg(LLM_MODEL, EMBEDDING_MODEL, LLM_HISTORY_LEN)  
# 使用 ChatGLM 的 readme 进行测试  
vector_store = init_knowledge_vector_store("/home/mw/project/test_chatglm_readme.md")  
```

其中vector_store的初始化可以传递 txt、docx、md 格式文件，或者包含md文件的目录。  

更多知识库加载方式可以参考[langchain文档](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/unstructured_file.html)，  
通过修改 init_knowledge_vector_store 方法进行兼容。  

**官方注**：ModelWhale GPU机型需要从云厂商拉取算力资源，耗时5~10min，且会预扣半小时资源价格的鲸币。如果资源未启动成功，预扣费用会在关闭编程页面后五分钟内退回，无需紧张，如遇问题欢迎[提报工单](https://www.heywhale.com/home/user/workorder)，客服会及时处理。  

## 更新日志  
[2023/05/11]  
1. 修复unstructured版本更新导致的PDF读取问题  

[2023/05/06]  
1. 更新 requirements。解决 langchain 版本导致的依赖冲突  
2. 同步上游代码结构，简化模块调用  
3. response 改为流式输出  
4. 真实案例知识库改为PDF格式  

[2023/04/28]  
1. 增加一个《动手学深度学习》书籍对话bot的真实案例  
2. 增加 ChineseTextSplitter 来避免召回内容过长导致的显存溢出问题  

In [2]:
# 安装依赖
!pip install -r requirements.txt -i https://mirror.sjtu.edu.cn/pypi/web/simple

Looking in indexes: https://mirror.sjtu.edu.cn/pypi/web/simple
     |████████████████████████████████| 600 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.7 MB 102.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 96.4 MB/s eta 0:00:01
     |████████████████████████████████| 19.2 MB 1.2 MB/s eta 0:00:011
     |████████████████████████████████| 1.5 MB 64.6 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 416 kB 85.4 MB/s eta 0:00:01
     |████████████████████████████████| 17.6 MB 86.7 MB/s eta 0:00:01
     |████████████████████████████████| 219 kB 84.2 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 75.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 

In [3]:
# 安装nltk_data
# 
!cp -r nltk_data /home/mw

In [4]:
# 使用 Markdown 格式打印模型输出
from IPython.display import display, Markdown, clear_output

def display_answer(agent, query, vs_path, history=[]):
    for resp, history in local_doc_qa.get_knowledge_based_answer(query=query,
                                                                 vs_path=vs_path,
                                                                 chat_history=history,
                                                                 streaming=True):
        clear_output(wait=True)
        display(Markdown(resp["result"]))
    return resp, history


In [5]:
import torch.cuda
import torch.backends

from configs import model_config

# 全局参数，修改后请重新初始化
model_config.embedding_model_dict = {
    "ernie-tiny": "nghuyong/ernie-3.0-nano-zh",
    "ernie-base": "nghuyong/ernie-3.0-base-zh",
    "text2vec-base": "shibing624/text2vec-base-chinese",
    "text2vec": "/home/mw/input/text2vec2538",
}
model_config.llm_model_dict = {
    "chatyuan": "ClueAI/ChatYuan-large-v2",
    "chatglm-6b-int4-qe": "THUDM/chatglm-6b-int4-qe",
    "chatglm-6b-int4": "THUDM/chatglm-6b-int4",
    "chatglm-6b-int8": "THUDM/chatglm-6b-int8",
    "chatglm-6b": "/home/mw/input/ChatGLM6B6449",
}
model_config.VS_ROOT_PATH = "/home/mw/temp"

from chains.local_doc_qa import LocalDocQA

EMBEDDING_MODEL = "text2vec" # embedding 模型，对应 embedding_model_dict
VECTOR_SEARCH_TOP_K = 6
LLM_MODEL = "chatglm-6b"     # LLM 模型名，对应 llm_model_dict
LLM_HISTORY_LEN = 3
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

local_doc_qa = LocalDocQA()

local_doc_qa.init_cfg(llm_model=LLM_MODEL,
                          embedding_model=EMBEDDING_MODEL,
                          llm_history_len=LLM_HISTORY_LEN,
                          top_k=VECTOR_SEARCH_TOP_K)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

No sentence-transformers model found with name /home/mw/input/text2vec2538. Creating a new one with MEAN pooling.


In [6]:
# 使用 ChatGLM 的 readme 进行测试
vs_path, _ = local_doc_qa.init_knowledge_vector_store("/home/mw/project/test_chatglm_readme.md")
vs_path

test_chatglm_readme.md 已成功加载
文件加载完毕，正在生成向量库


'/home/mw/temp/test_chatglm_readme_FAISS_20230506_170416'

In [7]:
# 测试未进行本地知识库接入时的结果
from IPython.display import display, Markdown, clear_output
for resp, history in local_doc_qa.llm._call("chatglm-6b 的局限性具体体现在哪里，如何实现改进"):
    clear_output(wait=True)
    display(Markdown(resp))

ChatGLM-6B 是一个基于 GPT-3 的模型，它的主要特点是可以处理自然语言文本数据，并能够进行语言生成。但是，ChatGLM-6B 仍然存在一些局限性，具体体现如下：

1. 模型容量限制：GPT-3 模型的训练需要大量计算资源，在实际应用中，模型容量有限，无法处理更复杂的自然语言任务。

2. 语言知识不足：GPT-3 模型是一种基于预训练的模型，其语言知识的深度和广度可能不足以处理所有自然语言任务，需要结合更多的语言知识和算法进行改进。

3. 模型稳定性不足：GPT-3 模型在训练过程中可能会出现不稳定的情况，尤其是在处理大规模数据时，需要不断改进模型的稳定性和鲁棒性。

为了改进 ChatGLM-6B 的局限性，可以考虑以下几个方面：

1. 利用更多的语言知识和算法：结合更多的语言知识和算法，如知识图谱、词向量、逻辑回归等，可以提高 ChatGLM-6B 的处理能力。

2. 使用更多的数据：使用更多的数据，可以提高 ChatGLM-6B 的训练效果和稳定性。可以使用更复杂的数据集，如新闻文章、学术论文等，来训练 ChatGLM-6B 模型。

3. 改进 GPT-3 模型：可以通过改进 GPT-3 模型，如使用更大的预训练数据集、使用更多的训练节点等，来提高 ChatGLM-6B 的处理能力。

4. 结合其他模型：可以结合其他模型，如 GLM、BERT、Transformer 等，来提高 ChatGLM-6B 的处理能力。可以将 ChatGLM-6B 和 other 模型进行融合，形成一个更复杂的模型。

5. 使用人工智能辅助技术：可以使用人工智能辅助技术，如自动化翻译、自然语言生成等，来改进 ChatGLM-6B 的应用范围和效率。

In [8]:
# 接入知识库后同一问题的返回结果
history = []
display_answer(local_doc_qa, query="chatglm-6b 的局限性具体体现在哪里，如何实现改进", vs_path=vs_path, history=history)

ChatGLM-6B 的局限性主要体现在以下几个方面：

1. 容量限制：ChatGLM-6B 模型的容量较小，只能存储有限的数据，因此在面对复杂的任务时，可能会出现生成错误或不准确的情况。

2. 知识储备不足：ChatGLM-6B 模型虽然具有一定的语言处理能力，但是其知识储备仍然有限，无法回答涉及到更深入或更高阶的问题。

3. 逻辑推理能力较弱：ChatGLM-6B 模型对于复杂的逻辑推理任务可能无法胜任，例如数学、编程等任务。

针对这些问题，可以采取以下改进措施：

1. 扩展知识储备：通过引入更多的语言模型和知识图谱，扩展 ChatGLM-6B 的知识储备，提高其回答正确率。

2. 提高容量：通过压缩量化模型、引入更多的训练数据等方式，提高 ChatGLM-6B 的容量，使其能够处理更复杂的任务。

3. 加强逻辑推理能力：通过引入逻辑推理模块或更高级的推理方法，提高 ChatGLM-6B 的逻辑推理能力，解决其推理能力不足的问题。

综上所述，针对 ChatGLM-6B 的局限性，需要采取多种改进措施，以提高其能力水平和解决特定问题的能力。

({'query': 'chatglm-6b 的局限性具体体现在哪里，如何实现改进',
  'result': 'ChatGLM-6B 的局限性主要体现在以下几个方面：\n\n1. 容量限制：ChatGLM-6B 模型的容量较小，只能存储有限的数据，因此在面对复杂的任务时，可能会出现生成错误或不准确的情况。\n\n2. 知识储备不足：ChatGLM-6B 模型虽然具有一定的语言处理能力，但是其知识储备仍然有限，无法回答涉及到更深入或更高阶的问题。\n\n3. 逻辑推理能力较弱：ChatGLM-6B 模型对于复杂的逻辑推理任务可能无法胜任，例如数学、编程等任务。\n\n针对这些问题，可以采取以下改进措施：\n\n1. 扩展知识储备：通过引入更多的语言模型和知识图谱，扩展 ChatGLM-6B 的知识储备，提高其回答正确率。\n\n2. 提高容量：通过压缩量化模型、引入更多的训练数据等方式，提高 ChatGLM-6B 的容量，使其能够处理更复杂的任务。\n\n3. 加强逻辑推理能力：通过引入逻辑推理模块或更高级的推理方法，提高 ChatGLM-6B 的逻辑推理能力，解决其推理能力不足的问题。\n\n综上所述，针对 ChatGLM-6B 的局限性，需要采取多种改进措施，以提高其能力水平和解决特定问题的能力。',
  'source_documents': [Document(page_content='ChatGLM-6BBlog • 🤗HF Repo • 🐦Twitter • 📃[GLM@ACL 22][GitHub] • 📃[GLM-130B@ICLR 23][GitHub]👋 加入我们的 Slack 和 WeChat介绍', metadata={'source': '/home/mw/project/test_chatglm_readme.md', 'page_number': 1, 'category': 'Title', 'score': 454.11597}),
   Document(page_content='不过，由于 ChatGLM-6B 的规模较小，目前已知其具有相当多的局限性，如事实性/数学逻辑错误，可能生成有害/有偏见内容，较弱的上下文能力，自我认知混乱，以及对英文指示生成与中文指示完全矛盾的内容。请大家在使用前了解这些问题，以免产生误解。更大的